# Packages Installs, imports, and presets

In [2]:
from scapy.all import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from collections import defaultdict
import os

In [3]:
TIMEOUT= 30
df_train= pd.read_parquet(f'data/iec104_train_{TIMEOUT}_v2.parquet')
df_validation = pd.read_parquet(f'data/iec104_validation_{TIMEOUT}_v2.parquet')
df_test=pd.read_parquet(f'data/iec104_test_{TIMEOUT}_v2.parquet')

In [4]:
assert df_train.groupby(df_train.index)['label'].nunique().max()==1

In [5]:
len(set(df_train.index))

331760

In [6]:
df_train

,,ip_version,ip_ihl,ip_tos,ip_len,ip_id,ip_flags,ip_frag,ip_ttl,ip_proto,ip_chksum,...,tcp_reserved,tcp_flags,tcp_window,tcp_chksum,tcp_urgptr,tcp_options,size,payload,payload_raw,label
flow,packet_number,,,,,,,,,,,,,,,,,,,,,
"[1, '192.168.1.20', '192.168.1.24', 2404, 50409, 'TCP', 'c_rp_na_1_DoS']",3483,4,5,0,56,3403,DF,0,128,6,27128,...,0,PA,254,45878,0,0,70,16,b'h\x0eP\x06f\x00\x01\x01\x03\x00\x01\x00\x01\...,1
"['192.168.1.19', '192.168.1.20', 2404, 3, 50050, 'TCP', 'm_sp_na_1_DoS']",8161,4,5,0,56,30061,DF,0,128,6,475,...,0,PA,255,7739,0,0,70,16,b'h\x0e^\xe7$\x00\x01\x01\x03\x00\x01\x00\x01\...,0
"[1, '192.168.1.20', '192.168.1.22', 2404, 49671, 'TCP', 'c_sc_na_1_DoS']",750,4,5,0,46,24595,DF,0,128,6,5948,...,0,PA,253,17467,0,0,60,6,b'h\x04\x83\x00\x00\x00',1
"[1, '192.168.1.20', '192.168.1.25', 2404, 50047, 'TCP', 'm_sp_na_1_DoS']",2152,4,5,0,72,26352,DF,0,128,6,4162,...,0,PA,252,48359,0,0,86,32,b'h\x0e^\x12\x06\x00\x01\x01\x03\x00\x01\x00\x...,0
"[1, '192.168.1.19', '192.168.1.29', 2404, 39873, 'TCP', 'c_rd_na_1_DoS']",5,4,5,0,58,32125,DF,0,128,6,63935,...,0,PA,260,42355,0,3,72,6,b'h\x04#\x00\x00\x00',8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"[1, '192.168.1.19', '192.168.1.29', 2404, 34591, 'TCP', 'c_rp_na_1_DoS']",5,4,5,0,58,25724,DF,0,64,6,21185,...,0,PA,502,33709,0,3,72,6,b'h\x04\x13\x00\x00\x00',9
"[1, '192.168.1.20', '192.168.1.21', 2404, 50705, 'TCP', 'c_rd_na_1_DoS']",579,4,5,0,46,17289,DF,0,128,6,13255,...,0,PA,255,62533,0,0,60,6,b'h\x04\x83\x00\x00\x00',1
"[1, '192.168.1.20', '192.168.1.24', 2404, 50046, 'TCP', 'm_sp_na_1_DoS']",2856,4,5,0,72,28561,DF,0,128,6,1954,...,0,PA,254,10801,0,0,86,32,b'h\x0eN\x19\x08\x00\x01\x01\x03\x00\x01\x00\x...,0


In [9]:
df_train.columns

Index(['ip_version', 'ip_ihl', 'ip_tos', 'ip_len', 'ip_id', 'ip_flags',
       'ip_frag', 'ip_ttl', 'ip_proto', 'ip_chksum', 'ip_src', 'ip_dst',
       'ip_options', 'time', 'tcp_sport', 'tcp_dport', 'tcp_seq', 'tcp_ack',
       'tcp_dataofs', 'tcp_reserved', 'tcp_flags', 'tcp_window', 'tcp_chksum',
       'tcp_urgptr', 'tcp_options', 'size', 'payload', 'payload_raw', 'label'],
      dtype='object')

In [11]:
from datetime import datetime
from scapy.all import IP, TCP, Raw, PacketList
import importlib
import my_flowmeter  # import the module here, so that it can be reloaded.
import warnings
from tqdm import tqdm

importlib.reload(my_flowmeter)
def create_scapy_packet(row):
    ip_layer = IP(
        version=row['ip_version'],
        ihl=row['ip_ihl'],
        tos=row['ip_tos'],
        len=row['ip_len'],
        id=row['ip_id'],
        flags=row['ip_flags'],
        frag=row['ip_frag'],
        ttl=row['ip_ttl'],
        proto=row['ip_proto'],
        chksum=row['ip_chksum'],
        src=row['ip_src'],
        dst=row['ip_dst'],
        options=row['ip_options']
    )

    tcp_layer = TCP(
        sport=row['tcp_sport'],
        dport=row['tcp_dport'],
        seq=row['tcp_seq'],
        ack=row['tcp_ack'],
        dataofs=row['tcp_dataofs'],
        reserved=row['tcp_reserved'],
        flags=row['tcp_flags'],
        window=row['tcp_window'],
        chksum=row['tcp_chksum'],
        urgptr=row['tcp_urgptr'],
        options=row['tcp_options']
    )

    # Payload can be either a string or raw bytes
    if row['payload_raw']:
        payload_layer = Raw(load=row['payload_raw'])


    # Combine the layers to form the packet
    packet = ip_layer / tcp_layer / payload_layer
    return packet

# def binary_vector_to_bytes(binary_vector):
#     """Convert a binary numpy vector to a bytes variable."""
#     return bytes(np.packbits(binary_vector))

def get_flows_statistics(df_input):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        fm = my_flowmeter.Flowmeter()
        flows_statistics = []
        fgb = df_input.groupby('flow')
        a = 0
        for i in tqdm(fgb):
            a += 1
            column, rows = i
            packets = []

            for index, row in rows.iterrows():
                packets.append(create_scapy_packet(row))

            packet_list = PacketList(packets)
            # Record the start time
            start_time = datetime.now()
            df = fm.build_feature_from_packet_list(packet_list).copy()
            # Record the end time
            end_time = datetime.now()
            # Calculate the duration
            duration = (end_time - start_time).total_seconds()

            label = rows['label'].iloc[0]
            df['label'] = label
            df['flow'] = column
            df['computation_time'] = duration
            flows_statistics.append(df)
            # display(df.head())
            # if a == 4:
            #     break
        return pd.concat(flows_statistics)

df_train_statistics = get_flows_statistics(df_train)
df_train_statistics.to_parquet('data/df_train_statistics.parquet')

df_validation_statistics = get_flows_statistics(df_validation)
df_validation_statistics.to_parquet('data/df_validation_statistics.parquet')

df_test_statistics = get_flows_statistics(df_test)
df_test_statistics.to_parquet('data/df_test_statistics.parquet')

  0%|                                                                                                                                                               | 0/13549 [00:00<?, ?it/s]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


  0%|                                                                                                                                                               | 0/13549 [00:08<?, ?it/s]


TypeError: While dissecting field 'options': 'int' object is not iterable